<img src="../../img/evaluation_data_preparation.png" alt="Evaluation data preparation" style="width: 100%; border-radius: 20px;"/>

In [21]:
%%HTML
<style>
    body {
        --vscode-font-family: "Itim"
    }
</style>

In [22]:
import sys
sys.path.append('../')
sys.path.append('../../')

import pandas as pd

from utils.data_preparation import *
from simon.utils.feature_generation import *

In [23]:
# Load and standardize validation data
path_validata = '../../../01_Data/datasets/validata_ornitho_ch_2023.csv'
date_format = '%d.%m.%Y'  # ch: '%d.%m.%Y'; de: %m/%d/%Y

# Data we need for data preparation
path_translator_names = '../../../01_Data/translators/translation_species_names_de_vs_ch.csv'
path_eea_grids = '../../../01_Data/shp_files/grids/eea_europe_grids_50km/inspire_compatible_grid_50km.shp'

# Load and standardize validation data
validata = pd.read_csv(path_validata, delimiter=get_delimiter(path_validata), low_memory=False)
validata = standardize_data(validata, 
                            date_format=date_format,
                            path_translator_species_names=path_translator_names,
                            eea_shapefile_path=path_eea_grids)
validata.total_count = validata.total_count.astype(float)
validata.altitude = validata.altitude.astype(float)

# Add land use feature according to CORINE Land Cover
validata = generate_land_use_clc(validata)
validata.to_csv('validata_ch_with_luse.csv')

In [67]:
# sort validata by id_validata
validata = validata.sort_values(by=['id_validata']).reset_index(drop=True)

In [68]:
# Load validata with ground truth
path_validata_with_ground_truth = '../../../01_Data/datasets/evaluation_data_with_gt_ch.csv'
validata_with_ground_truth = pd.read_csv(path_validata_with_ground_truth, delimiter=get_delimiter(path_validata_with_ground_truth), low_memory=False)

validata_with_ground_truth.columns = validata_with_ground_truth.columns.str.lower()
validata_with_ground_truth = validata_with_ground_truth.sort_values(by=['id_validata']).reset_index(drop=True)

ground_truth_information = validata_with_ground_truth[['id_validata', 'date_original', 'coord_lat_original', 'coord_lon_original', 'altitude_original', 'change_type']]

In [69]:
validata_merged = pd.merge(validata, ground_truth_information, on='id_validata', how='left')

In [80]:
# standardized validata with ground truth and land cover
validata_merged.to_csv('validata_ch_with_luse_and_gt.csv')